In [1]:
#Makes data paths
import os
import json
path = os.path.abspath('{}/../'.format('transfer_learning_text_classification.ipynb') )
data_path = '{}/data/mmimdb/dataset/'.format(path)
make_data_path = '{}/data/mmimdb/'.format(path)
import sys
sys.path.insert(0, make_data_path)
import make_data
import pandas as pd
import glob
from collections import Counter
import numpy as np
import matplotlib.pyplot as plt

#Prints example of movie data
with open('{}0399877.json'.format(data_path)) as json_data:
    data = json.load(json_data)
    print(data['genres'])
    print(data['title'])
    print(data['plot'])

['Documentary', 'Comedy', 'Drama', 'Fantasy', 'Mystery', 'Sci-Fi']
What the #$*! Do We (K)now!?
['"WHAT THE #$*! DO WE KNOW?!" is a radical departure from convention. It demands a freedom of view and greatness of thought so far unknown, indeed, not even dreamed of since Copernicus. It\'s a documentary. It\'s a story. It\'s mind-blowing special effects. This film plunges you into a world where quantum uncertainty is demonstrated - where neurological processes, and perceptual shifts are engaged and lived by its protagonist - where everything is alive, and reality is changed by every thought.', "Amanda, a divorced photographer, finds herself in a fantastic Alice-in-Wonderland experience when her daily, uninspired life literally begins to unravel, revealing the cellular, molecular and even quantum worlds which lie beneath. Guided by a Greek Chorus of leading scientists and mystics, she finds that if reality itself is not questionable, her notion of it certainly is. Stunning special effects

In [2]:
texts_df = pd.read_csv(make_data_path+"text_labels.csv")
texts_df.head()

,text,tags,tag count,plot_word_count
0,He Knows You're Alone A reluctant bride to be ...,Horror Thriller,2,101
1,Link Student Jane jobs as an assistant for the...,Horror,1,95
2,The Blacksmith Buster clowns around in a black...,Comedy,1,53
3,"Take the Lead In New York, the polite dance in...",Drama Music,2,316
4,Ping-pongkingen Rille is coming of age in a Sw...,Drama,1,149


In [3]:
genres = []
genre_count = {}

sys.path.insert(0, data_path)
for file in glob.glob('{}*.json'.format(data_path)):
    data = json.load(open(file))
    [genres.append(genre) for genre in data['genres']]

counts = Counter(genres)

In [4]:
counts_df = pd.DataFrame.from_dict(list(dict(counts).items()))
counts_named_df = counts_df.rename(columns={0:'Genre',1: '# Movies'})
counts_ordered_df = counts_named_df.sort_values(by='# Movies')

In [5]:
#to remove lower 25th percentile, also take out 'Film-Noir', 'Short', 'Sport'
genres_to_remove = ['Reality-TV', 
                    'Talk-Show', 
                    'Adult', 
                    'News']

In [6]:
texts_lower_genres_rm_df = texts_df.dropna()

In [7]:
texts_lower_genres_rm_df.loc[:,'tags'] = \
    texts_lower_genres_rm_df.tags.apply(
        lambda tags: ' '.join([tag for tag in tags.split() \
                               if tag not in genres_to_remove]))

/home/ubuntu/anaconda3/envs/gmu_bimodal/lib/python3.6/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [8]:
texts_no_dups_df = texts_lower_genres_rm_df.drop_duplicates(subset='text')

In [9]:
texts_tag_counts = texts_no_dups_df
texts_tag_counts.loc[:, 'tag_count'] = texts_tag_counts.tags.apply(
                                        lambda x: len(x.split(' ')))

In [10]:
texts_tag_word_counts = texts_tag_counts
texts_tag_word_counts.loc[:, 'plot_word_count'] = texts_tag_counts.text.apply(
                                        lambda x: len(x.split(' ')))

In [11]:
texts_tag_word_counts.to_csv("{}text_mod_labels.csv".format(make_data_path),index=False)

In [12]:
#Language Model
import torch
import fastai
from fastai.text import * 

In [13]:
#USE GPU - Device 0 
torch.cuda.set_device(0)

In [14]:
# Language model data
data_lm = TextLMDataBunch.from_csv(make_data_path, 'text_mod_labels.csv', text_cols = 'text')
data_lm.save()
data_lm = TextLMDataBunch.load(make_data_path) #TextLMDataBunch expects to find data_lm saved to a temp directory

In [15]:
# Language model creation with WT103
learn = language_model_learner(data_lm, pretrained_model=URLs.WT103_1, drop_mult=0.5)

In [16]:
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy
1,4.264611,3.905638,0.316027


In [17]:
# Unfreeze all layers and fit to MM-IMDB data (data_lm)
learn.unfreeze()
learn.fit_one_cycle(1, 1e-3)

epoch,train_loss,valid_loss,accuracy
1,3.854819,3.753877,0.333092


In [18]:
learn.predict("This movie really", n_words=15)

"This movie really did n't break in his pals visible here by telling the story of reaction from"

In [19]:
#Save the language model
learn.save_encoder('lm_encoder')

In [20]:
#Text Classifier
#Split data into train and validate sets
from sklearn.model_selection import train_test_split
texts_df = pd.read_csv(make_data_path+'text_mod_labels.csv')
train, valid = train_test_split(texts_df, test_size = 0.02, random_state = 0)

In [21]:
# Create classifier model data
data_multilabel = TextClasDataBunch.from_df(path, 
                                      train_df = train,
                                      valid_df = valid,
                                      text_cols = 'text', 
                                      label_cols ='tags',
                                      label_delim=' ', 
                                      vocab=data_lm.train_ds.vocab, 
                                      bs=32)
data_multilabel.save()
data_multilabel = TextClasDataBunch.load(path, bs=32)

In [22]:
learn = text_classifier_learner(data_multilabel, drop_mult=0.5, metrics = [accuracy_thresh, fbeta])
print("learn classifier defined. model summary:")
print(learn.model)
learn.load_encoder('{}/models/lm_encoder'.format(make_data_path))

learn classifier defined. model summary:
SequentialRNN(
  (0): MultiBatchRNNCore(
    (encoder): Embedding(37100, 400, padding_idx=1)
    (encoder_dp): EmbeddingDropout(
      (emb): Embedding(37100, 400, padding_idx=1)
    )
    (rnns): ModuleList(
      (0): WeightDropout(
        (module): LSTM(400, 1150, batch_first=True)
      )
      (1): WeightDropout(
        (module): LSTM(1150, 1150, batch_first=True)
      )
      (2): WeightDropout(
        (module): LSTM(1150, 400, batch_first=True)
      )
    )
    (input_dp): RNNDropout()
    (hidden_dps): ModuleList(
      (0): RNNDropout()
      (1): RNNDropout()
      (2): RNNDropout()
    )
  )
  (1): PoolingLinearClassifier(
    (layers): Sequential(
      (0): BatchNorm1d(1200, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (1): Dropout(p=0.2)
      (2): Linear(in_features=1200, out_features=50, bias=True)
      (3): ReLU(inplace)
      (4): BatchNorm1d(50, eps=1e-05, momentum=0.1, affine=True, track_running

In [23]:
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy_thresh,fbeta
1,0.247583,0.209090,0.910212,0.713383


In [24]:
learn.freeze_to(-2)
learn.fit_one_cycle(1, slice(5e-3/2., 5e-3))

epoch,train_loss,valid_loss,accuracy_thresh,fbeta
1,0.232806,0.196420,0.917823,0.740950


In [25]:
learn.unfreeze()
learn.fit_one_cycle(1, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy_thresh,fbeta
1,0.212172,0.191583,0.919653,0.740876
